<a href="https://colab.research.google.com/github/sophie55555/Quest/blob/main/jelly_pub_vgg16(stepdecay%2Cpretrained%3Dx).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import datetime
from PIL import Image
from tqdm import tqdm

In [16]:
train_dir = "/content/drive/MyDrive/Jellyfish/Train_Test_Valid/Train/"
valid_dir = "/content/drive/MyDrive/Jellyfish/Train_Test_Valid/valid/"
test_dir = "/content/drive/MyDrive/Jellyfish/Train_Test_Valid/test/"

In [17]:
import os

print("📂 Train 폴더:", os.listdir(train_dir))
print("📂 Valid 폴더:", os.listdir(valid_dir))
print("📂 Test 폴더:", os.listdir(test_dir))

📂 Train 폴더: ['compass_jellyfish', 'Moon_jellyfish', 'mauve_stinger_jellyfish', 'blue_jellyfish', 'lions_mane_jellyfish', 'barrel_jellyfish']
📂 Valid 폴더: ['compass_jellyfish', 'Moon_jellyfish', 'mauve_stinger_jellyfish', 'blue_jellyfish', 'lions_mane_jellyfish', 'barrel_jellyfish']
📂 Test 폴더: ['mauve_stinger_jellyfish', 'blue_jellyfish', 'lions_mane_jellyfish', 'barrel_jellyfish', 'compass_jellyfish', 'Moon_jellyfish']


In [18]:
import os
from PIL import Image
from tqdm import tqdm

# 변환된 데이터 저장 폴더 설정
output_base_dir = "/content/drive/MyDrive/Jellyfish/Jellyfish_Processed"
output_train_dir = os.path.join(output_base_dir, "Train")
output_valid_dir = os.path.join(output_base_dir, "valid")
output_test_dir = os.path.join(output_base_dir, "test")

# 변환할 크기 설정
target_size = (224, 224)

# 변환된 데이터 저장 폴더 생성
for path in [output_train_dir, output_valid_dir, output_test_dir]:
    os.makedirs(path, exist_ok=True)

# Train, Valid, Test 각각 변환 수행
datasets = {
    "Train": (train_dir, output_train_dir),
    "valid": (valid_dir, output_valid_dir),
    "test": (test_dir, output_test_dir),
}

for dataset_name, (input_dir, output_dir) in datasets.items():
    print(f"🚀 Processing {dataset_name} dataset...")

    for cls in tqdm(os.listdir(input_dir), desc=f"Processing {dataset_name}"):
        class_path = os.path.join(input_dir, cls)
        output_class_path = os.path.join(output_dir, cls)

        if os.path.isdir(class_path):  # 폴더인지 확인
            os.makedirs(output_class_path, exist_ok=True)

            for img_name in os.listdir(class_path):
                img_path = os.path.join(class_path, img_name)
                output_img_path = os.path.join(output_class_path, img_name)

                try:
                    img = Image.open(img_path).convert("RGB")  # RGB 변환
                    if img.size != target_size:
                        img = img.resize(target_size, Image.LANCZOS)  # 224x224 변환
                    img.save(output_img_path, "JPEG")  # 변환된 이미지 저장
                except Exception as e:
                    print(f"⚠️ 변환 실패: {img_name} - {e}")

    print(f"✅ {dataset_name} 데이터 224x224 변환 완료!\n")

🚀 Processing Train dataset...


Processing Train: 100%|██████████| 6/6 [00:23<00:00,  3.93s/it]


✅ Train 데이터 224x224 변환 완료!

🚀 Processing valid dataset...


Processing valid: 100%|██████████| 6/6 [00:01<00:00,  5.48it/s]


✅ valid 데이터 224x224 변환 완료!

🚀 Processing test dataset...


Processing test: 100%|██████████| 6/6 [00:01<00:00,  5.61it/s]

✅ test 데이터 224x224 변환 완료!



In [19]:
import os
from PIL import Image
from tqdm import tqdm

# 변환할 데이터 경로 설정 (224x224 변환된 데이터 기준)
processed_base_dir = "/content/drive/MyDrive/Jellyfish/Jellyfish_Processed"
train_dir = os.path.join(processed_base_dir, "Train")
valid_dir = os.path.join(processed_base_dir, "valid")
test_dir = os.path.join(processed_base_dir, "test")

# 확장자를 변환할 데이터셋
datasets = {
    "Train": train_dir,
    "valid": valid_dir,
    "test": test_dir,
}

# 변환 실행
for dataset_name, dataset_dir in datasets.items():
    print(f"🚀 Converting {dataset_name} dataset to .jpg...")

    for cls in tqdm(os.listdir(dataset_dir), desc=f"Converting {dataset_name}"):
        class_path = os.path.join(dataset_dir, cls)

        if os.path.isdir(class_path):
            for img_name in os.listdir(class_path):
                img_path = os.path.join(class_path, img_name)
                ext = os.path.splitext(img_name)[-1].lower()  # 확장자 추출

                # ✅ .jpg(소문자)만 유지, 나머지는 변환
                if ext not in [".jpg"]:
                    output_img_path = os.path.splitext(img_path)[0] + ".jpg"

                    try:
                        img = Image.open(img_path).convert("RGB")  # RGB 변환
                        img.save(output_img_path, "JPEG")  # .jpg로 저장
                        os.remove(img_path)  # 원본 파일 삭제
                    except Exception as e:
                        print(f"⚠️ 변환 실패: {img_name} - {e}")

    print(f"✅ {dataset_name} 데이터 확장자 .jpg 변환 완료!\n")

🚀 Converting Train dataset to .jpg...


Converting Train: 100%|██████████| 6/6 [00:03<00:00,  1.95it/s]


✅ Train 데이터 확장자 .jpg 변환 완료!

🚀 Converting valid dataset to .jpg...


Converting valid: 100%|██████████| 6/6 [00:00<00:00, 58.00it/s]


✅ valid 데이터 확장자 .jpg 변환 완료!

🚀 Converting test dataset to .jpg...


Converting test: 100%|██████████| 6/6 [00:00<00:00, 78.48it/s]

✅ test 데이터 확장자 .jpg 변환 완료!



In [20]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 데이터 경로 (전처리된 데이터 사용)
train_dir = "/content/drive/MyDrive/Jellyfish/Jellyfish_Processed/Train/"
valid_dir = "/content/drive/MyDrive/Jellyfish/Jellyfish_Processed/valid/"
test_dir = "/content/drive/MyDrive/Jellyfish/Jellyfish_Processed/test/"

# ✅ Train 데이터 증강 적용
train_datagen = ImageDataGenerator(
    rescale=1./255,  # 정규화 (0~1 범위)
    rotation_range=20,  # 랜덤 회전 (최대 ±20도)
    width_shift_range=0.2,  # 가로 이동 (최대 20%)
    height_shift_range=0.2,  # 세로 이동 (최대 20%)
    shear_range=0.2,  # 기울기 변환
    zoom_range=0.2,  # 확대/축소 변환 (최대 ±20%)
    horizontal_flip=True,  # 좌우 반전
    fill_mode="nearest"  # 빈 영역 보정 방식
)

# ✅ Validation & Test 데이터는 증강 없이 정규화만 적용
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# ✅ 데이터 로드
batch_size = 32  # 배치 크기 설정

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode="categorical"
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode="categorical"
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode="categorical"
)

# ✅ 클래스 이름 확인
print("✅ 클래스 이름:", train_generator.class_indices)

Found 900 images belonging to 6 classes.
Found 39 images belonging to 6 classes.
Found 40 images belonging to 6 classes.
✅ 클래스 이름: {'Moon_jellyfish': 0, 'barrel_jellyfish': 1, 'blue_jellyfish': 2, 'compass_jellyfish': 3, 'lions_mane_jellyfish': 4, 'mauve_stinger_jellyfish': 5}


In [21]:
from tensorflow.keras import layers, models

# ✅ VGG16 모델 정의
def build_vgg16(input_shape=(224, 224, 3), num_classes=6):
    model = models.Sequential()

    # Conv Block 1
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=input_shape))
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), strides=2))

    # Conv Block 2
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), strides=2))

    # Conv Block 3
    model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), strides=2))

    # Conv Block 4
    model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), strides=2))

    # Conv Block 5
    model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), strides=2))

    # Fully Connected Layers
    model.add(layers.Flatten())
    model.add(layers.Dense(4096, activation='relu'))
    model.add(layers.Dense(4096, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))  # 6개 클래스 분류

    return model

 Step Decay 함수 정의

In [48]:
import tensorflow as tf
from tensorflow.keras.callbacks import LearningRateScheduler, TensorBoard
import os
import datetime

# ✅ Step Decay 함수 정의
def step_decay(epoch):
    initial_lrate = 0.001  # 초기 학습률
    drop = 0.5  # 감소 비율 (50%)
    epochs_drop = 20  # 20 에폭마다 감소
    lrate = initial_lrate * (drop ** (epoch // epochs_drop))
    return lrate

# ✅ Step Decay가 정상적으로 작동하는지 확인
print("📌 Step Decay 학습률 변화 확인:")
for epoch in range(0, 121, 20):  # 0, 20, 40, 60, 80, 100, 120
    print(f"Epoch {epoch}: Learning Rate = {step_decay(epoch):.6f}")

# ✅ Step Decay 적용 (Callback 설정)
lr_callback = LearningRateScheduler(step_decay)

# ✅ TensorBoard 로그 저장 경로 설정
log_dir = os.path.join("logs", "fit", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# ✅ 기존에 정의된 VGG16 모델 사용
vgg16_model = build_vgg16()

# 모델 컴파일
vgg16_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),  # 초기 학습률 설정
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# ✅ 모델 학습 (120 에폭)
history = vgg16_model.fit(
    train_generator,
    epochs=120,  # 120 에폭 학습
    validation_data=valid_generator,
    callbacks=[lr_callback, tensorboard_callback]  # ✅ Step Decay + TensorBoard 적용
)

# ✅ 학습 후 test 데이터 평가 진행
test_loss, test_acc = vgg16_model.evaluate(test_generator)
print(f"🧪 Test Accuracy: {test_acc:.4f}")

📌 Step Decay 학습률 변화 확인:
Epoch 0: Learning Rate = 0.001000
Epoch 20: Learning Rate = 0.000500
Epoch 40: Learning Rate = 0.000250
Epoch 60: Learning Rate = 0.000125
Epoch 80: Learning Rate = 0.000063
Epoch 100: Learning Rate = 0.000031
Epoch 120: Learning Rate = 0.000016
Epoch 1/120


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


KeyboardInterrupt: 

 Step Decay 함수 정의 안한 코드

In [22]:
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
import os
import datetime

# ✅ 하이퍼파라미터 설정 (고정)
initial_learning_rate = 0.001  # 초기 학습률
num_epochs = 10  # 총 에폭 수

# ✅ TensorBoard 로그 저장 경로 설정
log_dir = os.path.join("logs", "fit", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# ✅ 기존에 정의된 VGG16 모델 사용
vgg16_model = build_vgg16()

# 모델 컴파일
vgg16_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=initial_learning_rate),  # 초기 학습률 설정
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# ✅ 모델 학습 (고정된 에폭 수)
history = vgg16_model.fit(
    train_generator,
    epochs=num_epochs,  # 고정된 에폭 수
    validation_data=valid_generator,
    callbacks=[tensorboard_callback]  # ✅ TensorBoard 적용
)

# ✅ 학습 후 test 데이터 평가 진행
test_loss, test_acc = vgg16_model.evaluate(test_generator)
print(f"🧪 Test Accuracy: {test_acc:.4f}")

# ✅ TensorBoard 실행 (코랩)
%load_ext tensorboard
%tensorboard --logdir logs/fit

Epoch 1/10
28/29 ━━━━━━━━━━━━━━━━━━━━ 1:10 70s/step - accuracy: 0.1565 - loss: 2.3322

KeyboardInterrupt: 